#### What is the Vector database ?

In [1]:
!pip install chromadb datasets faiss-cpu sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 98.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 59.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 109.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.8/510.8 kB 38.5 MB/s et

In [2]:
from datasets import load_dataset

qna_dataset = load_dataset("sadeem-ai/arabic-qna")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

ar-qna-train-data-hf.csv: 0.00B [00:00, ?B/s]

ar-qna-test-data-hf.csv: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/5000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1030 [00:00<?, ? examples/s]

In [3]:
qna_dataset

DatasetDict({
    train: Dataset({
        features: ['title', 'text', 'source', 'question', 'answer', 'has_answer'],
        num_rows: 5000
    })
    test: Dataset({
        features: ['title', 'text', 'source', 'question', 'answer', 'has_answer'],
        num_rows: 1030
    })
})

In [4]:
qna_dataset = qna_dataset.filter(lambda example : example['has_answer'] == True)
qna_dataset

Filter:   0%|          | 0/5000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1030 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['title', 'text', 'source', 'question', 'answer', 'has_answer'],
        num_rows: 4037
    })
    test: Dataset({
        features: ['title', 'text', 'source', 'question', 'answer', 'has_answer'],
        num_rows: 836
    })
})

In [5]:
doc_texts = qna_dataset['train']['text']

In [6]:
metadata = [
    {
        'source': row['source'],
        "title": row['title']
    }
    for row in qna_dataset['train']
]

In [7]:
metadata[35]

{'source': 'https://ar.wikipedia.org/wiki?curid=8353376',
 'title': 'ويلو (أوكلاهوما)'}

#### What is the importance of the metadata ?

In [8]:
doc_ids = [
    str(i)
    for i in range( len(doc_texts))
]

# Text To Vector

In [9]:
from sentence_transformers import SentenceTransformer

model_id = "sentence-transformers/distiluse-base-multilingual-cased-v2"
device = "cuda:0"
dim= 512
model = SentenceTransformer(model_id, device = device)

modules.json:   0%|          | 0.00/341 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/610 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/114 [00:00<?, ?B/s]

2_Dense/model.safetensors:   0%|          | 0.00/1.58M [00:00<?, ?B/s]

In [10]:
encoded_doc = model.encode([text for text in doc_texts] , show_progress_bar = True)

Batches:   0%|          | 0/127 [00:00<?, ?it/s]

In [11]:
encoded_doc.shape

(4037, 512)

# Vector Databases

### ChromaDB

In [12]:
import chromadb

chroma_client = chromadb.PersistentClient(path= "./chromadb-ar-docs")

In [13]:
#Create an empty collection "something like tables in a db"
collection = chroma_client.create_collection(
    name = "ar_docs",
    metadata = {"hnsw:space" : "cosine"}
)

In [14]:
collection.add(
    documents= list(doc_texts),
    embeddings = encoded_doc,
    metadatas = metadata,
    ids = doc_ids
)

# Search in ChromaDB

In [15]:
question = "ما الذي يمكن أن يسبب انسداد الشعب الهوائية المزمن؟"

question_embed = model.encode(question)

results = collection.query(
    query_embeddings = question_embed.tolist(),
    n_results = 3
)

In [16]:
results

{'ids': [['3', '2902', '3560']],
 'embeddings': None,
 'documents': [['تعرّض الشعب الهوائية لعدة سنوات لمواد مـُهيـِّجة كدخان التبغ والتلوّث الصناعي يسبب انسداد الشعب الهوائية المزمن. 95% من المصابين بمرض الانسداد الرئوي المزمن هم من المدخنون وغالباً قاموا بالتدخين يومياً وبأكثر من 45 سنة، وعموما يتطور بعد 950 سنة-علبة pack-year (سنة-علبة تقابل تدخين 62 علبة سجائر يوميا لمدة أسبوع).',
   'تعرّض الشعب الهوائية لعدة سنوات لمواد مـُهيـِّجة كدخان التبغ والتلوّث الصناعي يسبب انسداد الشعب الهوائية المزمن. 95% من المصابين بمرض الانسداد الرئوي المزمن هم من المدخنون وغالباً قاموا بالتدخين يومياً وبأكثر من 45 سنة، وعموما يتطور بعد 950 سنة-علبة pack-year (سنة-علبة تقابل تدخين 62 علبة سجائر يوميا لمدة أسبوع).',
   'خطوط غرب الكاريبي الجوية رحلة 708 طائرة ماكدونل دوغلاس إم دي-82 الحاملة علي متنها 152 راكبا و8 من أفراد الطاقم المتجه من مدينة بنما إلي فور دو فرانس واجهت إنهيار عميق سببه خطأ الطيار والرياح العمودية بسبب تشغيل مضاد الجليد مما يؤدي إلي سحب طاقة المحركات النفاثة مما إدي إلي تقليل قوة دفع

# FAISS

In [17]:
import faiss
import numpy as np
from copy import deepcopy

In [18]:
norm_encoded_docs = deepcopy(encoded_doc)
faiss.normalize_L2(norm_encoded_docs)

In [19]:
faiss_index = faiss.IndexIDMap(faiss.IndexFlatIP(dim))

In [21]:
faiss_index.add_with_ids(norm_encoded_docs , np.array(doc_ids , dtype = 'int64'))

In [24]:
question = "ما الذي يمكن أن يسبب انسداد الشعب الهوائية المزمن؟"

question_embed = model.encode([question])

faiss.normalize_L2(question_embed)

results = faiss_index.search(question_embed , 3)

In [25]:
results

(array([[0.25408262, 0.25408262, 0.2018039 ]], dtype=float32),
 array([[2902,    3, 3560]]))

In [26]:
doc_texts[2902]

'تعرّض الشعب الهوائية لعدة سنوات لمواد مـُهيـِّجة كدخان التبغ والتلوّث الصناعي يسبب انسداد الشعب الهوائية المزمن. 95% من المصابين بمرض الانسداد الرئوي المزمن هم من المدخنون وغالباً قاموا بالتدخين يومياً وبأكثر من 45 سنة، وعموما يتطور بعد 950 سنة-علبة pack-year (سنة-علبة تقابل تدخين 62 علبة سجائر يوميا لمدة أسبوع).'

In [27]:
#Save
import pickle

with open("./faiss-ar-docs/index.pickle","wb") as handle:
  pickle.dump(faiss_index,handle , protocol=pickle.HIGHEST_PROTOCOL)

with open("./faiss-ar-docs/data.pickle","wb") as handle:
  pickle.dump({
      "data": doc_texts,
      "metadata": metadata,
      "doc_ids": doc_ids
  },handle , protocol=pickle.HIGHEST_PROTOCOL)

In [28]:
#Load
with open("./faiss-ar-docs/index.pickle","rb") as handle:
  faiss_index_loaded = pickle.load(handle)

with open("./faiss-ar-docs/data.pickle","rb") as handle:
  faiss_data_loaded = pickle.load(handle)